# Building Fast Queries on a Laptop Store Inventory CSV  File

In this project, we will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory. **Our goal is to create a class that represents our inventory.** The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:
* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

The mock inventory data we'll use is stored in the file, `laptops.csv`, which is adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices).

Here is a brief description of the columns:

| Column | Description |
|-|-|
| ID | A unique identifier for the laptop. |
| Company | The name of the company that produces the laptop. |
| Product | The name of the laptop. |
| TypeName | The type of laptop. |
| Inches | The size of the screen in inches. |
| ScreenResolution | The resolution of the screen. |
| CPU | The laptop CPU. |
| RAM | The amount of RAM in the laptop. |
| Memory | The size of the hard drive. |
| GPU | The graphics card name. |
| OpSys | The name of the operating system. |
| Weight | The laptop weight. |
| Price | The price of the laptop. |

Let's explore the data and print the first few rows before we start implementing a way to represent this data as the store inventory.

In [1]:
# Import libraries we'll use
import time
import random
import contextlib
from csv import reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Show graphs in notebook
%matplotlib inline

# Formatting for graphs
sns.set(style='white')
plt.style.use('dark_background')

# Read in data as a list of lists to work with later, separating data as rows and header
with open('laptops.csv') as file:
    rows = list(reader(file))
    header = rows[0]
    rows = rows[1:]

# Read in data as a pandas dataframe for readability
rows_pd = pd.read_csv('laptops.csv')

# Explore data
rows_pd.info()
rows_pd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price             1303 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 132.5+ KB


,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,6571244,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339
1,7287764,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898
2,3362737,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575
3,9722156,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537
4,8550527,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803


## 1. Creating an `Inventory` class object

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in it.

In [2]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of strings
            
# Instantiate inventory object, then print headers and number of enttires (rows) in the data
inventory = Inventory('laptops.csv')
print(inventory.header, '\n')
print('Number of entries in inventory:', len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'] 

Number of entries in inventory: 1303


## 2. Adding laptop id look-up functionality to the `Inventory` class

Throughout this project we'll be making several improvements to the `Inventory` class. We'll copy and paste the last version of the class to modify and keep track of changes.

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named `get_laptop_from_id()`. This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [3]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of strings
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

In [4]:
# Instantiate inventory object, then test laptop id look-up method
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### 2.1 Optimizing the laptop id look-up method

The algorithm above requires us to look at every single row to find the id that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, you have to imagine that this code could be used in situations where the inventory contains millions of rows. Also, if we perform a lot of queries, even on a small dataset, the slow query performance will start to add up. It might eventually become the bottleneck of the application.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the `get_laptop_from_id()` method. We can do this by:

* Preprocess the data and create the dictionary in the `__init__()` method.
* Re-implement the `get_laptop_from_id()` method. We will do it as a new method so that we can compare the two.

In [5]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

In [6]:
# Instantiate inventory object, then test fast laptop id look-up method
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### 2.2 Comparing laptop id look-up method performance

The `get_laptop_from_id()` method has time complexity O(R) where R is the number of rows. In contrast, the new implementation as time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the `random` module. Then, use both methods to lookup these same IDs. We will use the `time` module to measure the execution time of each lookup and, for each method, add all times together.

In [7]:
# Context manager for timing code execution
@contextlib.contextmanager
def timer():
    """Time the execution of a context block.

    Yields:
      runtime (float): The execution time.
    """
    start = time.time()
    
    # Send control back to the context block
    yield
    
    end = time.time()
    print('        Elapsed time: {:.5f} seconds'.format(end - start))

In [8]:
# Generate a list of 10,000 random id numbers for testing
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

# Instantiate Inventory class
inventory = Inventory('laptops.csv')

# Time the execution of both methods, displaying results
print('Total time to look up 10,000 id numbers:')
print('    get_laptop_from_id() method:')
with timer():
    for identifier in ids:
        inventory.get_laptop_from_id(identifier)
        
print('    get_laptop_from_id_fast() method:')
with timer():
    for identifier in ids:
        inventory.get_laptop_from_id_fast(identifier)

Total time to look up 10,000 id numbers:
    get_laptop_from_id() method:
        Elapsed time: 0.98310 seconds
    get_laptop_from_id_fast() method:
        Elapsed time: 0.00344 seconds


Our times:
* get_laptop_from_id() method:
        Elapsed time: 0.989 seconds
* get_laptop_from_id_fast() method:
        Elapsed time: 0.003 seconds

The method referencing a dictionary of id numbers **ran over 300 times faster than looping through all rows of data.** That's a huge difference!

## 3. Laptop promotion price look-up

Sometimes, the store offers a promotion where we give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory:

The prices of these laptops are \\$1,339, \\$898, and \\$575. Say we offered a gift card of \\$2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is \\$2,237 (\\$1,339 plus \\$898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full \\$2,500.

We don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

We'll Write a method that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [9]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> Inventory.get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        for row1 in self.rows:
            if row1[-1] == dollars:
                return True
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False

In [10]:
# Instantiate inventory object, then test laptop promo price look-up method
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


### 3.1 Optimizing the laptop promotion price look-up method

Since we only care about whether or not there is a matching dollar amount in the inventory, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time, O(1), whether there is a laptop with a given price.

Let's implement this to make our code run faster.

In [11]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.prices.add(row[-1])
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> Inventory.get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        for row1 in self.rows:
            if row1[-1] == dollars:
                return True
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False

In [12]:
# Instantiate inventory object, then test fast laptop promo price look-up method
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


### 3.2 Comparing laptop promotion price look-up methods

The `check_promotion_dollars()` method has a quadratic time complexity, O(R^2), where R is the number of rows in the inventory dataset. This is because of the use of a nested for loop to check prices and their pair sums.

On the other hand, the `check_promotoin_dollars_fast()` method iterates over a set of prices once. Therefore, this method should perform faster since it has a constant time complexity, O(1).

Let's compare the two methods now.

In [13]:
# Generate a list of 100 random prices between $100 and $5,000
prices = [random.randint(100, 5000) for _ in range(100)]

# Instantiate Inventory class
inventory = Inventory('laptops.csv')

# Time execution of both laptop promo price look-up methods and display results
print('Total time to look-up 5,000 prices:')
print('    check_promotion_dollars() method:')
with timer():
    for price in prices:
        inventory.check_promotion_dollars(price)
        
print('    check_promotion_dollars_fast() method')
with timer():
    for price in prices:
        inventory.check_promotion_dollars_fast(price)

Total time to look-up 5,000 prices:
    check_promotion_dollars() method:
        Elapsed time: 2.55898 seconds
    check_promotion_dollars_fast() method
        Elapsed time: 0.00097 seconds


Our times:

* check_promotion_dollars() method:
        Elapsed time: 1.26032 seconds
* check_promotion_dollars_fast() method
        Elapsed time: 0.00049 seconds
        
The method referencing a set of prices **ran over 2000 times faster than the method running a nested loop** to check all combinations of prices. What a massive difference!

## 4. Using binary search to find laptops within a budget

Binary search is an algorithm used to find an element in a sorted list quickly. We'll leverage and extend that algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: 
* Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars.

We'll sort the inventory by price in the `__init__()` method and use the sorted inventoy in our new binary search method.

In [14]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.prices.add(row[-1])
            self.id_to_row[row[0]] = row
        self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> Inventory.get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        for row1 in self.rows:
            if row1[-1] == dollars:
                return True
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, budget):
        """Find the index of the first row whose price is higher than budget.
        
        Args:
            budget (int): The price budget.
            
        Returns:
            int: The index of the first row whose price is higher than the budget.
            
        Example:
            >>> find_first_laptop_more_expensive(300)
            60
        """
        index_start = 0
        index_end = len(self.rows_by_price) -1
        
        while index_start < index_end:
            index_middle = (index_end + index_start) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price > budget:
                index_end = index_middle
            else:
                index_start = index_middle + 1
                
        median_price = self.rows_by_price[index_start][-1]
        if median_price < budget:
            return -1
        return index_start

In [15]:
# Instantiate Inventory class and test binary search method
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


### 4.1 Is this faster than pandas?

Let's check which method is faster to find all laptops in the inventory below budget:
* Binary search on the Inventory class
* Pandas

An index is not very useful without applying it to the display the appropriate data.
Pandas will automatically display the filtered data we want below budget.
Therefore, we'll add an extra step after the `find_first_laptop_more_expensive()` to display the appropriate rows for our experiment.

In [16]:
# Generate a list of 1,000 random budgets between $100 and $3,000
budgets = [random.randint(100, 3000) for _ in range(1000)]

# Time the execution of both methods and display the results
print('Time to look-up laptops priced under 500 different budgets:')
print('    find_first_laptop_more_expensive() method:')
with timer():
    for budget in budgets:
        index_budget = inventory.find_first_laptop_more_expensive(budget)
        inventory.rows_by_price[:index_budget] # Display all rows below or equal to budget

print('    Using pandas:')
with timer():
    for budget in budgets:
        rows_pd[rows_pd['Price'] <= budget]

Time to look-up laptops priced under 500 different budgets:
    find_first_laptop_more_expensive() method:
        Elapsed time: 0.00864 seconds
    Using pandas:
        Elapsed time: 0.70458 seconds


Our times:

* find_first_laptop_more_expensive() method:
        Elapsed time: 0.01477 seconds
* Using pandas:
        Elapsed time: 0.93148 seconds
        
Wow, using our custom Inventory class method to display laptops under or equal to budget requirements ran **over 60 times faster than using pandas** to do the same.

While the difference isn't much if dealing with just one or a few customers, the difference would certainly add up when dealing with tens or hundreds of thousands of customers at once.

### 4.2 Finding laptops within price range

We'll modify our binary search method to search by price range rather than a single budget amount.

In [17]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.prices.add(row[-1])
            self.id_to_row[row[0]] = row
        self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> Inventory.get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        for row1 in self.rows:
            if row1[-1] == dollars:
                return True
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, budget):
        """Find the index of the first row whose price is higher than budget.
        
        Args:
            budget (int): The price budget.
            
        Returns:
            int: The index of the first row whose price is higher than the budget.
            
        Example:
            >>> find_first_laptop_more_expensive(300)
            60
        """
        index_start = 0
        index_end = len(self.rows_by_price) -1
        
        while index_start < index_end:
            index_middle = (index_end + index_start) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price > budget:
                index_end = index_middle
            else:
                index_start = index_middle + 1
                
        median_price = self.rows_by_price[index_start][-1]
        if median_price < budget:
            return -1
        return index_start
    
    def laptops_by_price_range(self, min_price, max_price):
        """Find the indices of rows whos price is in between the given max and min prices
        
        Args:
            min_price (int): The minimum price
            max_price (int): The maximum price
            
        Returns:
            tuple: A tuple of integers containing the indices corresponding to the first price
                   above or equal to the min_price, and the first price above or equal to
                   the max_price.
            
        Example:
            >>> laptops_by_price_range(500,1500)
            (245, 998)
        """
        try:
            index_start = next(i for i, row in enumerate(self.rows_by_price) 
                                   if row[-1] >= min_price)
        except StopIteration:
            return 'No laptops in that price range.'
        
        try:    
            index_end = next(i for i, row in enumerate(self.rows_by_price) 
                                 if row[-1] >= max_price)
        except StopIteration:
            index_end = -1
        
        if index_end == 0:
            return "No laptops in that price range."
            
        return index_start, index_end

In [18]:
# Instantiate Inventory class and test laptops by price range method
inventory = Inventory('laptops.csv')
print(inventory.laptops_by_price_range(600,800))
print(inventory.laptops_by_price_range(2000,7000))
print(inventory.laptops_by_price_range(-3,7000))
print(inventory.laptops_by_price_range(15000,20000))
print(inventory.laptops_by_price_range(600,500))
print(inventory.laptops_by_price_range(100,0))

(330, 509)
(1166, -1)
(0, -1)
No laptops in that price range.
(330, 245)
No laptops in that price range.


### 4.1 Optimizing search for laptops within price range

We'll try to make the code run faster next

In [19]:
class Inventory():
    """Represents the laptop store inventory.
    
    Args:
        csv_filename (str): A csv file name.
        
    Attributes:
        header (list): Column header names.
        rows (list): A list of lists of laptops and their specifications,
                              i.e. the laptop inventory.
                              
    """
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            rows = list(reader(file))
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            row[-1] = int(row[-1]) # csv.reader() reads each row as a list of string
            self.prices.add(row[-1])
            self.id_to_row[row[0]] = row
        self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> Inventory.get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        """Look-up and return row containing laptop ID number
        
        Args:
            laptop_id (str) : Unique laptop ID number
            
        Returns:
            list: The row containing the laptop ID number.
                  Will return None if there is not match.
                  
        Example:
            >>> get_laptop_from_id(6571244)
            ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 
            'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', 
            '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        for row1 in self.rows:
            if row1[-1] == dollars:
                return True
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        """Check whether one or two added up laptop prices equal a dollar amount
        
        Args:
            dollars (int): Dollar amount
            
        Returns:
            bool: True if the dollar amount is equal to one or two added up prices in the data
                  Otherwise False.
    
        Example:
            >>> check_promotion_dollars(442)
            False
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, budget):
        """Find the index of the first row whose price is higher than budget.
        
        Args:
            budget (int): The price budget.
            
        Returns:
            int: The index of the first row whose price is higher than the budget.
            
        Example:
            >>> find_first_laptop_more_expensive(300)
            60
        """
        index_start = 0
        index_end = len(self.rows_by_price) -1
        
        while index_start < index_end:
            index_middle = (index_end + index_start) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price > budget:
                index_end = index_middle
            else:
                index_start = index_middle + 1
                
        median_price = self.rows_by_price[index_start][-1]
        if median_price < budget:
            return -1
        return index_start
    
    def laptops_by_price_range(self, min_price, max_price):
        """Find the indices of rows whos price is in between the given max and min prices
        
        Args:
            min_price (int): The minimum price
            max_price (int): The maximum price
            
        Returns:
            tuple: A tuple of integers containing the indices corresponding to the first price
                   above or equal to the min_price, and the first price above or equal to
                   the max_price.
            
        Example:
            >>> laptops_by_price_range(500,1500)
            (245, 998)
        """
        try:
            index_start = next(i for i, row in enumerate(self.rows_by_price) 
                                   if row[-1] >= min_price)
        except StopIteration:
            return 'No laptops in that price range.'
        
        try:    
            index_end = next(i for i, row in enumerate(self.rows_by_price) 
                                 if row[-1] >= max_price)
        except StopIteration:
            index_end = -1
        
        if index_end == 0:
            return "No laptops in that price range."
            
        return index_start, index_end
    
    def laptops_by_price_range_fast(self, min_price, max_price):
        """Find the indices of rows whos price is in between the given max and min prices
        
        Args:
            min_price (int): The minimum price
            max_price (int): The maximum price
            
        Returns:
            tuple: A tuple of integers containing the indices corresponding to the first price
                   above or equal to the min_price, and the first price above or equal to
                   the max_price.
            
        Example:
            >>> laptops_by_price_range(500,1500)
            (245, 998)
        """
        if max_price <= 0:
            return 'No laptops in that price range.'
        
        index_min = 0
        index_end = len(self.rows_by_price) -1
    
        while index_min < index_end:
            index_middle = (index_min + index_end) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price == min_price:
                index_min = index_middle
                break
            elif median_price > min_price:
                index_end = index_middle
            else:
                index_min = index_middle + 1
                
        binary_min_price = self.rows_by_price[index_min][-1]
        if binary_min_price < min_price:
            return 'No laptops in that price range.'

        index_max = 0
        index_end = len(self.rows_by_price) -1
        
        while index_max < index_end:
            index_middle = (index_max + index_end) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price == max_price:
                index_max = index_middle
                break
            elif median_price > max_price:
                index_end = index_middle
            else:
                index_max = index_middle + 1

        binary_max_price = self.rows_by_price[index_max][-1]
        if binary_max_price < max_price:
            index_max = -1
            
        return index_min, index_max

In [20]:
    def laptops_by_price_range_fast(self, min_price, max_price):
        """Find the indices of rows whos price is in between the given max and min prices
        
        Args:
            min_price (int): The minimum price
            max_price (int): The maximum price
            
        Returns:
            tuple: A tuple of integers containing the indices corresponding to the first price
                   above or equal to the min_price, and the first price above or equal to
                   the max_price.
            
        Example:
            >>> laptops_by_price_range(500,1500)
            (245, 998)
        """
        if max_price == 0:
            return 'No laptops in that price range.'
        
        index_min = 0
        index_end = len(self.rows_by_price) -1
    
        while index_min < index_end:
            index_middle = (index_min + index_end) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price == min_price:
                index_min = index_middle
                break
            elif median_price > min_price:
                index_end = index_middle
            else:
                index_min = index_middle + 1
                
        binary_min_price = self.rows_by_price[index_min][-1]
        if binary_min_price < min_price:
            return 'No laptops in that price range.'

        index_max = 0
        index_end = len(self.rows_by_price) -1
        
        while index_max < index_end:
            index_middle = (index_max + index_end) // 2
            median_price = self.rows_by_price[index_middle][-1]
            if median_price == max_price:
                index_max = index_middle
                break
            elif median_price > max_price:
                index_end = index_middle
            else:
                index_max = index_middle + 1

        binary_max_price = self.rows_by_price[index_max][-1]
        if binary_max_price < max_price:
            index_max = -1
            
        return index_min, index_max

In [21]:
# Instantiate Inventory class and test fast laptops by price range method
inventory = Inventory('laptops.csv')
print(inventory.laptops_by_price_range_fast(600,800))
print(inventory.laptops_by_price_range_fast(2000,7000))
print(inventory.laptops_by_price_range_fast(-3,7000))
print(inventory.laptops_by_price_range_fast(15000,20000))
print(inventory.laptops_by_price_range_fast(600,500))
print(inventory.laptops_by_price_range_fast(100,0))

(330, 509)
(1166, -1)
(0, -1)
No laptops in that price range.
(330, 245)
No laptops in that price range.


### 4.2 Comparing methods for finding laptops between price ranges

We'll time both price range methods and compare them both.

In [22]:
prices = [(random.randint(200,1000), random.randint(500,2000)) for _ in range(1000)]

print('Total time to look-up laptops between 1000 different price ranges:')

print('    laptops_by_price_range() method:')
with timer():
    for price in prices:
        index = inventory.laptops_by_price_range(price[0], price[1])
        
print('    laptops_by_price_range_fast() method:')
with timer():
    for price in prices:
        index = inventory.laptops_by_price_range_fast(price[0], price[1])

Total time to look-up laptops between 1000 different price ranges:
    laptops_by_price_range() method:
        Elapsed time: 0.10330 seconds
    laptops_by_price_range_fast() method:
        Elapsed time: 0.01095 seconds


Our times:

* laptops_by_price_range() method:
        Elapsed time: 0.17543 seconds
* laptops_by_price_range_fast() method:
        Elapsed time: 0.01342 seconds

Once again, the binary search version of the method is faster, running **over 10 times faster than using using for loops.**



## Conclusion

We built a class, `Inventory`, to represent the inventory of our hypothetical laptop store. Within the class we added methods for a few useful queries, such as:
* Laptop unique ID look-up
* Promo price validation
* Filtering the inventory by budget
* Filtering the inventory by price range

We also optimized the time complexity of these algorithms to run anywhere from 10 to even 2000 times faster than their original implementations.
### Next steps

We could add further query functionality, such as filtering the inventory by RAM, CPU, GPU or any combination of the other column attributes.